In [ ]:
import sklearn
import sklearn.metrics
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas
import mplhep
import pickle
import awkward
import glob
import bz2
import os
import tqdm
import fastjet
import vector
import uproot
from pathlib import Path

import pickle
from functools import reduce
import mplhep
import boost_histogram as bh

mplhep.style.use("CMS")

import sys
sys.path += ["../../mlpf/"]
sys.path += ["../../mlpf/plotting/"]

from plot_utils import ELEM_LABELS_CMS, ELEM_NAMES_CMS
from plot_utils import CLASS_LABELS_CMS, CLASS_NAMES_CMS, EVALUATION_DATASET_NAMES
from plot_utils import cms_label, sample_label
from plot_utils import pid_to_text
from plot_utils import save_img

In [ ]:
def add_results(d0, d1):
    d_ret = {}
    k0 = set(d0.keys())
    k1 = set(d1.keys())

    for k in k0.intersection(k1):
        d_ret[k] = d0[k] + d1[k]

    for k in k0.difference(k1):
        d_ret[k] = d0[k]

    for k in k1.difference(k0):
        d_ret[k] = d1[k]

    return d_ret

In [ ]:
files = [pickle.load(open(fn, "rb")) for fn in glob.glob("/media/joosep/data/mlpf/plots_cms//out*.pkl")]
ret = reduce(add_results, files, {})

sample_keys = sorted(set(["/".join(k.split("/")[0:2]) for k in ret.keys() if not k.startswith("combined")]))
sample_keys_combined = sorted(set(["/".join(k.split("/")[0:3]) for k in ret.keys() if k.startswith("combined")]))

In [ ]:
sample_keys

In [ ]:
sample_keys_combined

In [ ]:
for k in sorted(ret.keys()):
    print(k)

In [ ]:
sample_labels = {
    "QCDForPF_14TeV_TuneCUETP8M1_cfi": r"QCD $p_T \in [15, 3000]\ \mathrm{GeV}$",
    "ZTT_All_hadronic_14TeV_TuneCUETP8M1_cfi": r"$\mathrm{Z}\rightarrow \mathrm{\tau}\mathrm{\tau}$",
    "TTbar_14TeV_TuneCUETP8M1_cfi": r"$\mathrm{t}\overline{\mathrm{t}}$+jets",
}

pu_labels = {
    "nopu": "no pileup",
    "pu55to75": "PU 55-75",
}

def sample_label(ax, sample, additional_text=""):
    spl = sample.split("/")
    pu_name = spl[-2]
    sample_name = spl[-1]
    plt.text(0.03, 0.97, sample_labels[sample_name] + "\n" + pu_labels[pu_name] + "\n" + additional_text, ha="left", va="top", transform=ax.transAxes)

In [ ]:
for sample in sample_keys_combined:
    plt.figure()
    ax = plt.axes()
    mplhep.histplot(ret[f"{sample}/particles_pt_pythia"], label="Pythia")
    mplhep.histplot(ret[f"{sample}/particles_pt_caloparticle"], label="CaloParticle")
    mplhep.histplot(ret[f"{sample}/particles_pt_target"], label="Target")
    mplhep.histplot(ret[f"{sample}/particles_pt_target_pumask"], label="Target, PU mask", ls="--")
    plt.xscale("log")
    plt.yscale("log")
    plt.legend(loc=1)
    plt.ylim(1, 1e8)
    cms_label(ax)
    sample_label(ax, sample)
    plt.xlabel("particle $p_T$ [GeV]")
    save_img("{}_particles_pt.png".format(sample.replace("/", "_")), cp_dir=Path("./"))

In [ ]:
for pid in [11, 22, 211, 130]:
    for sample in sample_keys_combined:
        plt.figure()
        ax = plt.axes()
        mplhep.histplot(ret[f"{sample}/particle_{pid}_pt_pythia"], label="Pythia")
        mplhep.histplot(ret[f"{sample}/particle_{pid}_pt_caloparticle"], label="CaloParticle")
        mplhep.histplot(ret[f"{sample}/particle_{pid}_pt_target"], label="Target")
        mplhep.histplot(ret[f"{sample}/particle_{pid}_pt_target_pumask"], label="Target, PU mask", ls="--")
        plt.xscale("log")
        plt.yscale("log")
        plt.legend(loc=1)
        plt.ylim(1, 1e7)
        cms_label(ax)
        sample_label(ax, sample, str(pid))
        plt.xlabel("particle $p_T$ [GeV]")
        save_img("{}_particle_{}_pt.png".format(sample.replace("/", "_"), pid), cp_dir=Path("./"))

In [ ]:
for sample in sample_keys:
    plt.figure()
    ax = plt.axes()
    mplhep.histplot(ret[f"{sample}/jets_pt_genjet"], label="Pythia")
    mplhep.histplot(ret[f"{sample}/jets_pt_cand"], label="PF")
    mplhep.histplot(ret[f"{sample}/jets_pt_target"], label="Target")
    mplhep.histplot(ret[f"{sample}/jets_pt_target_pumask"], label="Target, PU mask", ls="--")
    plt.xscale("log")
    plt.legend()
    cms_label(ax)
    sample_label(ax, sample)
    plt.yscale("log")
    plt.ylim(1,1e8)
    plt.xlabel("jet $p_T$ [GeV]")
    save_img("{}_jet_pt.png".format(sample.replace("/", "_")), cp_dir=Path("./"))

In [ ]:
rebin = 5
for sample in sample_keys:
    plt.figure()
    ax = plt.axes()
    mplhep.histplot(0.0*ret[f"{sample}/jets_pt_ratio_cand"][bh.rebin(rebin)], yerr=False)
    mplhep.histplot(ret[f"{sample}/jets_pt_ratio_cand"][bh.rebin(rebin)], yerr=False, label="PF")
    # mplhep.histplot(ret[f"{sample}/jets_pt_ratio_caloparticle"][bh.rebin(rebin)], yerr=False, label="CaloParticle")
    mplhep.histplot(ret[f"{sample}/jets_pt_ratio_target"][bh.rebin(rebin)], yerr=False, label="Target")
    mplhep.histplot(ret[f"{sample}/jets_pt_ratio_target_pumask"][bh.rebin(rebin)], yerr=False, label="Target, PU mask", ls="--")
    plt.legend()
    cms_label(ax)
    sample_label(ax, sample)
    plt.yscale("log")
    plt.ylim(1,1e8)
    plt.xlabel("jet response $p_T/p_{T,Pythia}$")
    save_img("{}_jet_response.png".format(sample.replace("/", "_")), cp_dir=Path("./"))

In [ ]:
rebin = 1
for sample in sample_keys:
    plt.figure()
    ax = plt.axes()
    mplhep.histplot(ret[f"{sample}/met_pythia"][bh.rebin(rebin)], yerr=False, label="Pythia")
    mplhep.histplot(ret[f"{sample}/met_cand"][bh.rebin(rebin)], yerr=False, label="PF")
    mplhep.histplot(ret[f"{sample}/met_target"][bh.rebin(rebin)], yerr=False, label="Target")
    mplhep.histplot(ret[f"{sample}/met_target_pumask"][bh.rebin(rebin)], yerr=False, label="Target, PU mask", ls="--")
    plt.legend(loc=1)
    plt.yscale("log")
    plt.xscale("log")
    cms_label(ax)
    sample_label(ax, sample)
    plt.ylim(1,1e8)
    plt.xlabel("MET [GeV]")
    save_img("{}_met.png".format(sample.replace("/", "_")), cp_dir=Path("./"))